<img src = "https://drive.google.com/uc?export=view&id=1FSCcyEY8_AsxSOTiv88txhmxbUwfLw_P" alt = "Encabezado MLDS" width = "100%">  </img>

# **Visualización de mapas coropléticos con *Plotly* y *Folium*** 
---

<img src="https://images.prismic.io/plotly-marketing-website/bd1f702a-b623-48ab-a459-3ee92a7499b4_logo-plotly.svg?auto=compress,format" alt="scipy" width="50%">
<img src="https://leafletjs.com/docs/images/logo.png" alt="statsmodels" width="45%">

En este último taller guiado discutiremos dos librerías que permiten construir visualizaciones de datos espaciales y, en particular, visualizaciones de **mapas coropléticos** en *Python*. La particularidad que tienen las herramientas que revisaremos aquí es que permiten construir **mapas  interactivos**:

1. Se discutirá nuevamente **[*Plotly*](https://plot.ly/python/reference/#choropleth)** y sus funcionalidades especializadas en datos geoespaciales. 
2. Se presentará **[*Folium*](https://python-visualization.github.io/folium/)**, una librería de *Python* que permite usar las funcionalidades de **[*Leaflet.js*](https://leafletjs.com/)**, una librería de *JavaScript* especializada en la construcción de mapas interactivos.

# **0. Datos utilizados**
---
En este material utilizaremos varios *dataset* externos para construir las visualizaciones de ejemplo. Para cargar estos datos de una manera rápida y sin interactuar con el gestor de archivos de *Google Colab* le recomendamos simplemente ejecutar los siguientes comandos para realizar la descarga. Estos comandos solo sirven con **Google Colaboratory**, o en su defecto, en local si está trabajando en un sistema operativo *Linux*.

**Ejecute la siguiente celda sin modificarla y compruebe que se hayan descargado los archivos:**

In [ ]:
# Descargar los archivos utilizados en este material.
!pip install gdown
#Ejecute esta celda para cargar en el entorno los archivos utilizados en el transcurso de este taller.
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1qv93EbbIvgleCU9k9UKDdfiTD8ijby1Z' -O 2011_US_AGRI_Exports
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Zu_cyyUoEUFTfn5exQ-rz1IaVyhoH9Ws' -O 2014_World_GDP
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=18HnuITDegAYw5eaUsx969HgBa-v96Zvv' -O Movilidad_bogota_2015.csv
# Mapa de Colombia
!gdown https://drive.google.com/uc?id=1wMwLcKZ0v18Dwse0Ln1-GTimFHpgSlir 

!ls

# **1. Mapas coropléticos interactivos con *Plotly***
---
Como se explicó en el taller de *GeoPandas*, un **mapa coroplético** es un mapa en el cual las regiones se dibujan con diferentes tonos de color que van de acuerdo a cierta estadística representativa, como por ejemplo su población, rangos de ingreso, etc. En *Plotly* se pueden crear **mapas coropléticos interactivos** con gran facilidad. Además, se pueden **exportar a archivos HTML**, manteniendo su interactividad. 

A continuación, estudiaremos diferentes formas de hacerlo.

## **1.1. Importar y configurar *Plotly***
---

Es necesario instalar *Plotly* y *GeoPandas*:

In [ ]:
!pip install -U plotly        # Instalamos Plotly (trabajaremos con la versión más reciente).
!pip install -U geopandas     # Instalamos GeoPandas (importante para cargar las formas de los mapas de ejemplo).

Importamos las librerías básicas de análisis y visualización de datos:

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 110   

Importamos *Plotly* y las utilidades principales usadas en este material. 

In [ ]:
import plotly
import plotly.graph_objs as go 
import plotly.express as px

import json

Para conocer las versiones de todas las librerías ejecute la siguiente celda:


In [ ]:
# Versiones de Python y demás librerías utilizadas.
!python --version

print('NumPy', np.__version__)
print('Pandas', pd.__version__)
print('Matplotlib', mpl.__version__)
print('GeoPandas', gpd.__version__)
print('Plotly', plotly.__version__)

Este material se realizó con las siguientes versiones:
* *Python*: 3.7.10
* *NumPy*:  1.19.5
* *Pandas*:  1.1.5
* *Matplotlib*:  3.2.2
* *GeoPandas*: 0.9.0
* *Plotly*: 4.14.3

## **1.2 Mapas predefinidos**
---
*Plotly* cuenta con una selección de datos geográficos disponibles para la construcción de visualizaciones. Para generar los objetos necesarios para renderizar el mapa deseado tenemos que comenzar a construir una traza de tipo **`go.Choropleth`** con la información relevante. Algunos de los argumentos más importantes de este constructor son:

* **`locationmode`** = Modo de ubicación. Puede ser uno de los siguientes: **`'ISO-3', 'USA-states', 'country names', 'geojson-id'`**. Vienen predefinidos en *Plotly*.
* **`locations = <Lista de ubicaciones>`**: Con este argumento se declaran las ubicaciones que se desean considerar de la ubicación general. Depende de lo que se haya seleccionado en **`locationmode`**.

* **`colorscale = <colores>`**: Para definir los colores a usar en la visualización podemos optar por varias opciones:
  * Usar una cadena de texto predefinida con el nombre de un *colormap* de *Plotly*. Por ejemplo, cadenas como **`'viridis', 'spectral', 'blues' 'rainbow'`**, entre [otras](https://plotly.com/python/builtin-colorscales/).
  * Crear una escala de colores [personalizada](https://plot.ly/python/heatmap-and-contour-colorscales/).

* **`text = <Lista de etiquetas>`**: Arreglo con el texto para mostrar en cada punto.
* **`z = <Lista de valores z>`**: Arreglo con los valores a representar en el mapa coroplético. Es decir, la profundidad percibida del área definida en el argumento **`locations`**.

Por ejemplo:

In [ ]:
# Creamos una traza de tipo Choropleth.

choropleth = go.Choropleth(locationmode = 'ISO-3',                                    # Modo de ubicación. ISO-3 es el valor predefinido en Plotly.
                           locations = ['COL','CHL','PER','BRA','ARG', 'VEN'],        # Lista de ubicaciones como códigos de países. 
                           colorscale= 'ylgnbu',                                      # Escala de color a usar.   
                           text= ['Colombia','Chile','Perú',                          # Texto mostrado al pasar el mouse por encima.   
                                 'Brasil', 'Argentina', 'Venezuela'],             
                           z=[30, 50, 21, 40],                                        # Valor o magnitud representada.
                            colorbar = {'title': 'Escala de magnitud representada'})  # Configuración de la barra de color.

# Creamos el atributo data de la figura de Plotly con el contenido de la traza.
data = [choropleth]

Una vez definido el diccionario de configuración, se configura el  objeto con los aspectos de estructura y forma del mapa (**`layout`**). En él, se define el argumento **`geo`**, que define los parámetros usados en la construcción de visualizaciones geográficas. Algunos de estos son:

* **`scope`**: Define el alcance geográfico de la visualización generada. Puede ser uno de los siguientes: 
  > **`'world', 'usa', 'europe', 'asia', 'africa', 'north america', 'south america'`**.

* **`projection`**: La proyección cartográfica usada en la generación de la gráfica. Este es a su vez otro diccionario y su tipo puede ser definido por el argumento **`type`**. Además, se puede definir la escala o *zoom* inicial de la figura con su argumento **`scale`**.

* **`center`**: Centro por defecto de la visualización. Este argumento es un diccionario que acepta los argumentos **`lat`** y **`lon`** para definir los valores de latitud y longitud.

* Otros argumentos de configuración y visibilidad de elementos geográficos, como los siguientes:
  * Visibilidad, color y grosor de las líneas costeras (**`showcoastlines, coastlinecolor, coastlinewidth`**).
  * Visibilidad, color y grosor de los ríos (**`showrivers, rivercolor, riverwidth`**).
  * Visibilidad, color y grosor de las líneas de división de países (**`showcountries, countrycolor, countrywidth`**).
  * Visibilidad y color del territorio (**`showland, landcolor`**).
  * Visibilidad y color del oceano (**`showocean, oceancolor`**).
  * Visibilidad y color de los lagos (**`showlakes, lakecolor`**).

  Para conocer en detalle más argumentos de utilidad consulte la siguiente [página](https://plotly.com/python/reference/layout/geo/) de la documentación oficial.

En este caso usaremos el mapa predefinido en *Plotly* para América del sur. Así:


In [ ]:
layout = go.Layout(geo = {'scope':'south america',                                 # Alcance geográfico. En este caso representamos unicamente a américa del sur.
                     'projection': dict(type = 'equirectangular', scale = 1),   # Tipo y escala de la proyección.
                     'center': dict(lat = 4.6097, lon = -74.0817),            # Coordenadas del centro. En este caso usamos la latitud y longitud de Bogotá.
                     'showocean': True, 'countrycolor': 'red'                 # Configuraciones de estilo adicionales.                
              },
                width = 1200, height = 600,                                    # Argumentos generales del layout.   
              )

Finalmente, se construye la figura pasando como parámetros los dos diccionarios creados anteriormente, y visualizamos el mapa:

In [ ]:
mapa = go.Figure(data=data, layout=layout)

mapa.show()

Nótese la **interactividad** del mapa coroplético. Puede usar el cursor y el *scroll* para interactuar con el mapa. 




Adicionalmente, las gráficas generadas dentro del *notebook* también se pueden exportar a una **página html** con el método **`write_html`**. Esto facilita enormemente la portabilidad de las visualizaciones que construya sin perder la interactividad. Una vez exporte sus visualizaciones a **HTML** podrá abrirlas en un navegador web, sin depender de Google Colaboratory ni de Python. 

Para exportar:

In [ ]:
# Generamos el archivo HTML con el mapa creado.
mapa.write_html('sur_america.html')

Esto crea el archivo **`sur_america.html`** en el sistema de archivos. Lo invitamos a que descargue este archivo y, posteriormente, lo abra con su navegador para ver la versión web de la visualización generada.

**Estados Unidos**
***

Para hacer otro ejemplo, usaremos datos reales de los estados de Estados Unidos para generar un mapa coroplético con la información de las [exportaciones agrícolas del año 2011](https://www.kaggle.com/prakashkumar27/2011-us-agricultural-exports-by-states) de *Kaggle*, preparado para realizar este tipo de visualizaciones.

Verifique que esté cargado el archivo **`'2011_US_AGRI_Exports'`.** De lo contrario, vaya a la Sección **0. Datos utilizados** y ejecute la celda para cargar los archivos.


In [ ]:
!ls 

Cargamos el archivo en un DataFrame de *Pandas*:

In [ ]:
df = pd.read_csv('2011_US_AGRI_Exports')
df.head()

El *dataset* viene con un campo especial **`text`**, que almacena un resumen de los datos en formato de texto *html*, ideal para representarlo en nuestra visualización. Por ejemplo:

In [ ]:
from IPython.display import display_html, HTML     # Módulos de IPython para la visualización de código HTML.

display_html(HTML(df.loc[0]['text']))              # Mostramos el contenido HTML de uno de los valores usados.

Ahora construimos el diccionario de datos con algunos argumentos adicionales para los marcadores:

In [ ]:
usa_trace = go.Choropleth(locations = df['code'],            # Columna donde se encuentran los códigos de estados.
                          locationmode = 'USA-states',       # Predefinido en Plotly para el scope de USA.
                          text = df['text'],                 # Columna donde se encuentra el texto HTML a representar.
                          z = df['total exports'],           # Columna donde tenemos el dato que vamos a representar en el mapa (total de exportaciones).
                          marker = dict(                     # Argumento de configuración de los marcadores.
                                    line = {
                                            'color' : 'rgb(255,255,255)', # Marcadores de color blanco.
                                            'width' : 1                   # Grosor del marcador.
                                          }),
                          colorbar = {                       # Argumento de configuración de la barra de colores.
                                      'title':"Millones USD"              # Título de la barra de colores.
                                    },
                          colorscale = 'Bluered_r'           # Paleta/escala de colores.
                          ) 

# Atributo data de la figura de Plotly.
data = [usa_trace]

Además, construimos el *layout* con más argumentos:

In [ ]:
layout = dict(title = 'Exportaciones agrícolas de USA en 2011 por Estado',
              geo = dict(scope='usa',                   # Alcance geográfico de Estados Unidos.
                         showlakes = True,              # Permitimos la visualización de lagos en la geografía mostrada.
                         lakecolor = 'rgb(85,173,240)') # Color de los lagos.  (Un tono de azul claro)
             )

Creamos la figura:

In [ ]:
mapa = go.Figure(data = data, layout = layout)
mapa.show()

Si pasa el mouse sobre algún estado, se muestra el texto con el resumen de los valores correspondientes a cada estado. Además, podemos ver cómo mediante una simple configuración se muestran en la parte superior izquierda los grandes lagos que limitan con Canadá.

**Mapa mundial**
***

Para este ejemplo, utilizaremos datos con el Producto Interno Bruto (**GDP** del inglés *Gross Domestic Product*) de las naciones del mundo en billones de dólares registrado en el año 2014.


In [ ]:
# Cargamos el DataFrame con la información requerida. Verifique que el archivo esté cargado en el entorno.

df = pd.read_csv('2014_World_GDP')
df.head()

In [ ]:
data = go.Choropleth(        
        locations = df['CODE'],            # Códigos por país usados en la ubicación del scope mundial.
        z = df['GDP (BILLIONS)'],          # Valor representado por la escala de color. Usamos el GDP nacional.
        text = df['COUNTRY'],              # Texto mostrado al pasar el mouse. En este caso mostramos el nombre del país.
        colorbar = {                       # Configuración (título) de la barra de colores.
            'title' : 'GDP Billones USD'   
            },
        colorscale = 'ylgnbu'              # Paleta/escala de color usada.
      ) 

En este caso definimos la [proyección cartográfica](https://es.wikipedia.org/wiki/Proyecci%C3%B3n_cartogr%C3%A1fica) del mapa, la cual determina la representación del globo. Puede tomar cualquiera de los siguientes valores: 
*  **`equirectangular`**
*  **`mercator`**
*  **`orthographic`**
*  **`natural earth`**
*  **`kavrayskiy7`**
*  **`miller`**
*  **`robinson`**
*  **`eckert4`**
*  **`azimuthal equal area`**
*  **`azimuthal equidistant`**
*  **`conic equal area`**
*  **`conic conformal`**
*  **`conic equidistant`**
*  **`gnomonic`**
*  **`stereographic`**
*  **`mollweide`**
*  **`hammer`**
*  **`transverse mercator`**
*  **`albers usa`**
*  **`winkel tripel`**
*  **`aitoff`**
*  **`sinusoidal`** 

Lo invitamos a probar los distintos parámetros reemplazando el valor del argumento **`projection`** del diccionario **`layout`**. Para más información, consulte la [documentación](https://plotly.com/python/map-configuration/#map-projections).

In [ ]:
layout = dict(
    title = '2014 Global GDP',
    geo = dict(
        # No es necesario configurar el "scope". Por defecto scope='world".
        projection = {'type':'orthographic'},      # Pruebe otras opciones de la lista de arriba.
        showocean = True,
        oceancolor = 'rgb(85,173,240)'            # Color del océano.
    ),
    width = 800
)

In [ ]:
# Mostramos el mapa generado.
mapa = go.Figure(data = data, layout = layout)
mapa.show()

Pruebe arrastrar el mapa con el clic izquierdo presionado para ver la información de otros países.

## **1.3. Cargando un mapa nuevo**
---
A continuación, cargaremos el mapa de Colombia con la división política a nivel de departamentos. Para esto debemos cargar la información geográfica para realizar la representación específica de nuestra tarea.

> **Nota**: El archivo **`colombia.zip`** se descargó desde la **Sección 0** de este taller.

Ejecute la siguiente instrucción para descomprimir el archivo:



In [ ]:
!unzip colombia.zip

Cuando haya descomprimido el archivo, usaremos *GeoPandas* para crear el *GeoDataFrame* con la división política de los Departamentos en Colombia:

In [ ]:
col_deps = gpd.read_file(r"shapes/Limite Departamental.shp") # Información geográfica de los departamentos de Colombia.

Este *dataset* contiene información geográfica muy detallada de las fronteras de cada departamento. Para evitar problemas de rendimiento en la gráfica generada, se recomienda simplificar la geometría de su objeto *GeoDataFrame*. Para esto, puede utilizar el método **`simplify`**, que recibe un valor de tolerancia sobre el cuál generar la nueva geometría simplificada.

In [ ]:
col_deps['geometry'] = col_deps['geometry'].simplify(1e-2)

Mostramos la cabecera del *GeoDataFrame* que contiene:
* **`COD_DEPART`**: código del Departamento.
* **`COUNT`**: número de municipios en el Departamento.
* **`Nombre`**: nombre del Departamento o Distrito.
* **`geometry`**: geometría (polígono) del mapa

In [ ]:
col_deps.head()

Para usar este mapa desde *Plotly*, tenemos que convertirlo a formato **`geojson`** mediante la función **`to_json()`**. Luego, se cargan los datos mediante la librería **`json`** al objeto **`col_json`**. Este objeto será utilizado para informarle a *Plotly* cuáles son las formas que debe representar en el mapa posteriormente.

In [ ]:
# Usamos json.loads para almacenar un objeto en este formato a partir de un GeoDataFrame.

col_json = json.loads(col_deps.to_json())

Para entender la nueva representación de los datos, mostraremos el primer elemento:

In [ ]:
# Convertimos la información del GeoDataFrame al formato JSON.

print(col_json['features'][0])

Se trata de un diccionario que contiene entre otras cosas: una llave **`properties`**, donde están los datos de los departamentos; y una llave **`geometry`**, donde se almacena la geometría del mapa de ese departamento.

A continuación, creamos y visualizamos el mapa coroplético con los **departamentos** de Colombia, mostrando el número de municipios de cada departamento. Para esto utilizamos la función de *Plotly Express* **`px.choropleth`**.

> **Nota**: La ejecución puede tardar unos minutos antes de generar los resultados.

In [ ]:
col = px.choropleth(col_deps,                           # El GeoDataFrame/DataFrame donde están los datos
                    geojson = col_json,                 # GeoJSON obtenido a partir de la geometría del GeoDataFrame.
                    color="COUNT",                      # Nombre de la columna del dataframe que queremos representar en el mapa coroplético
                    locations="Nombre",                 # Nombre de la columna del dataframe que coincide con las localizaciones
                    featureidkey="properties.Nombre",   # Llave dentro del GeoJSON usada como referencia.  
                    color_continuous_scale  = "Agsunset",
                    projection="mercator",
                    labels={'COUNT':'Número de<br>Municipios'} # Como queremos que aparezca la información
)
col.update_geos(fitbounds="locations", visible=False) # Para que aparezca centrado donde están los datos (en Colombia)
col.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
col.show()

Finalmente, exportamos el mapa a HTML:

In [ ]:
col.write_html("colombia.html")

Descargue el mapa exportado y ábralo en el navegador web como cualquier página web tradicional.

## **1.4. Usando *Mapbox***
---

En esta sección presentaremos los mapas de **_Mapbox_**, una de las funcionalidades avanzadas para la construcción de mapas de *Plotly*. 

Inicialmente, procedemos a crear la interacción con ***Mapbox***, un servicio en línea para el acceso a datos geográficos a partir de una consulta definida en el lado del cliente. Es posible utilizar diferentes capas para los mapas.

1. Si desea puede crear una cuenta en [**mapbox**](https://www.mapbox.com/).
2. En caso de crear su cuenta, *mapbox* generará un token de uso personal. En nuestro ejemplo usaremos un **token** de prueba, usado para la demostración.

> **Nota:** El token de prueba podría quedar inactivo o ser deshabilitado. Si esto ocurre, se sugiere a cada estudiante crear su propio ***TOKEN DE ACCESO*** en [**mapbox**](https://www.mapbox.com/).

![](http://c1.staticflickr.com/1/580/23252769485_032fe65b95_h.jpg)

**Colombia: población en algunas ciudades**
***
Para este ejemplo usaremos unos datos recopilados manualmente de varias ciudades de Colombia, en donde se tiene información de la posición geoespacial, el nombre, la población estimada y el área total en kilómetros cuadrados. Nótese que en este ejemplo no necesitamos tener disponibles las formas del mapa para poder trabajar, para eso usaremos *Mapbox*.

In [ ]:
# Lista de datos de algunas ciudades en Colombia

data = [['Bogotá', 4.6097102, -74.081749, 8264029, 1587], 
        ['Cali', 3.4372201, -76.5224991, 2434110, 562], 
        ['Medellín', 6.2518401, -75.563591, 2522081, 380],
        ['Barranquilla', 10.9685402, -74.7813187, 1232226, 166],
        ['Cartagena de Indias', 10.3997202, -75.5144424, 1006323, 709], 
        ['Cúcuta', 7.8939099, -72.5078201, 652320, 1176],
        ['Bucaramanga', 7.1253901, -73.1197968, 522439, 162],
        ['Pereira', 4.8133302, -75.6961136, 406348, 702],
        ['Santa Marta', 11.2407904, -74.1990433, 499219, 2448], 
        ['Ibagué', 4.43889, -75.2322235, 543564, 1439],
        ['Pasto', 1.2136101, -77.2811127, 386598, 1131], 
        ['Manizales', 5.0688901, -75.5173798, 373862, 571],
        ['Neiva', 2.9273, -75.2818909, 329462, 1553]]

df_col = pd.DataFrame(data, columns = ['Ciudad', 'Lat', 'Lon', 'Población', 'Área_total_km2']) 
df_col

Construiremos un mapa de burbuja de Colombia donde el **tamaño** del marcador sea dado por la población.

Para esto, crearemos una traza de tipo **`go.Scattermapbox`** de la siguiente forma:

In [ ]:
# Atributo data del mapa.

scatter_mapbox = go.Scattermapbox(
                    lon = df_col['Lon'],   # Longitud de los marcadores.
                    lat = df_col['Lat'],   # Latitud de los marcadores.
                    marker = dict(
                        size = 20 + df_col['Población']/200000,              # Tamaño del marcador a partir del valor reescalado de la población.                                                               
                    ),
                    text=df_col['Ciudad']  # Texto mostrado al pasar el mouse. En este caso es simplemente el nombre de la ciudad.
                )

data = [scatter_mapbox]

El siguiente es un token de prueba de *Mapbox*. Recuerde que puede obtener uno personal registrándose como se indicó arriba.

In [ ]:
#Token de prueba de Mapbox. Puede crear una cuenta y usar su propio token reemplazando este código.
token='pk.eyJ1IjoiZmVyZXN0cmVwb2NhIiwiYSI6ImNqdHQ4Zzc4MTE5MDA0NG1zeXlwMHBmZjMifQ.gdi4f1MAJor5u5_YWGCzOw' 

Ahora definimos el *layout* de la gráfica. En él se debe indicar el *token* de *Mapbox*.

In [ ]:
# Detalles del layout.

layout = go.Layout(
        title = 'Ciudades de Colombia',
        #Configuración interna de Mapbox
        mapbox = dict( 
            accesstoken = token,              # Token de acceso de Mapbox
            center= dict(lat=4.6, lon=-74.0), # Coordenada donde centrar el mapa inicialmente (Bogotá aprox.).
            zoom=5.2,                         # Acercamiento inicial del mapa.
        ),
        width = 1000,
        height = 1000
    )

In [ ]:
## Generación de la figura
fig = go.Figure(data=data, layout=layout)
fig.show()

Aunque este mapa es muy sencillo, en este ejemplo se puede apreciar algo muy interesante de trabajar con los mapas de *Mapbox*: intente hacer *zoom in* en cualquier ciudad de su interés y vea el nivel de detalle que se podría alcanzar al trabajar con un servicio en la nube.

**Colombia: mapa de municipios**
***
Ahora generaremos un mapa coroplético con la función **`px.choroplethmapbox`**, análoga a la función **`px.choropleth`** discutida previamente, pero apoyada por la funcionalidad de *Mapbox*. Para esto, utilizaremos un *dataset* de la geometría de los municipios de Colombia para representar la división política de Antioquia, el departamento con más municipios del país.

Para empezar, cargaremos el *GeoDataFrame*:

In [ ]:
col_mun = gpd.read_file(r"shapes/Limite Municipal.shp") # Municipios

col_mun.head()

Realizaremos selección condicional para obtener únicamente los municipios del departamento de Antioquia.

In [ ]:
col_mun = col_mun[col_mun['DEPARTAMEN']  == 'ANTIOQUIA']

Al igual que antes, simplificaremos la geometría para mejorar el rendimiento.

In [ ]:
col_mun['geometry'] = col_mun['geometry'].simplify(1e-3)

Creamos el *GeoJSON* correspondiente:

In [ ]:
# Usamos json.loads para almacenar un objeto en este formato a partir de un GeoDataFrame.

mun_json = json.loads(col_mun.to_json())

Para indicarle a *Mapbox* el centro de nuestra visualización, generamos el centroide de la unión de todos los municipios:

In [ ]:
center = col_mun.loc[:, 'geometry'].unary_union.centroid.coords[0]

center

Vamos a codificar el área (en Km$^2$) con ayuda del color y generar la visualización centrada

In [ ]:
col = px.choropleth_mapbox(col_mun,                          # El GeoDataFrame/DataFrame donde están los datos
                    geojson = mun_json,                      # GeoJSON obtenido a partir de la geometría del GeoDataFrame.
                    color="AREA_KM",                         # Nombre de la columna del dataframe que queremos representar en el mapa coroplético, en este caso por área.
                    locations="NOMBRE_ENT",                  # Nombre de la columna del dataframe que coincide con las localizaciones
                    featureidkey="properties.NOMBRE_ENT",    # Llave dentro del GeoJSON usada como referencia.  
                    opacity = 0.5,                           # Definimos un valor de opacidad para las áreas dibujadas.   
                    color_continuous_scale  = "Agsunset",    # Paleta/escala de color usada.
                    center={"lon": center[0],                # Centro definido con el centroide del departamento.
                            "lat": center[1]},
                    mapbox_style="carto-positron")           # Estilo de visualización de mapbox.

col.update_geos(fitbounds="locations", visible=False) # Para que aparezca centrado donde están los datos (en Colombia)
col.update_layout(margin={"r":0,"t":0,"l":0,"b":0}, height = 600)
col.show()

# **2. Mapas coropléticos con Leaflet y Folium**
---

*Leaflet* es una librería de código abierto escrita en *JavaScript* y diseñada con el objetivo de producir mapas interactivos compatibles con dispositivos móviles. Por su parte, *Folium* es una librería de *Python* que sirve de *wrapper* de *Leaflet* para *Python*, es decir, sirve de puente entre *Python* y *Leaflet*.

### **2.1. Importar y configurar *Folium***
---
Para empezar, instalamos *Folium* mediante **`pip`**.

In [ ]:
# Instalamos folium
!pip install folium

Ahora importamos en nuestro entorno de ejecución las librerías necesarias:

In [ ]:
# Importamos folium y algunos plug-ins adicionales que usaremos.
import folium
from folium.plugins import FastMarkerCluster
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

Este taller guiado fue desarrollado con la versión de **Folium: 0.8.3**.

In [ ]:
# Verificamos la versión de Folium a utilizar
print(f'Folium: {folium.__version__}')

### **2.2. Datos: Movilidad en Bogotá**
---
Para los ejemplos creados a continuación usaremos algunos datos de movilidad en la ciudad de Bogotá. Estos datos corresponden a la caracterización de la movilidad en Bogotá en el año 2015 realizada por la Secretaría Distrital de Movilidad y tomada de la página de Datos Abiertos Colombia disponible en el siguiente [enlace](https://www.datos.gov.co/Transporte/Encuesta-de-movilidad-de-Bogot-2015-Caracterizaci-/mvbb-bn7j).

> **Nota:** El archivo correspondiente se cargó en las celdas de la sección 0 de esta guía.


In [ ]:
# Cargamos los datos con los que trabajaremos en un DataFrame de Pandas.
data = pd.read_csv("Movilidad_bogota_2015.csv")

Antes de iniciar, realizaremos una exploración inicial de los datos a nuestra disposición.

In [ ]:
# Información acerca del DataFrame.
data.info()

In [ ]:
# Observamos las primeras filas.
data.head()

#### **2.2.1 Entendimiento y preparación de los datos**
---
Antes de empezar, debemos corregir algunos detalles. Primero, eliminamos los registros con valores faltantes:

In [ ]:
# Eliminamos los valores faltantes.
data.dropna(inplace=True, 
            how="any",
            subset=["LATITUD_ORIGEN", "LATITUD_DESTINO", "LONGITUD_ORIGEN", "LONGITUD_DESTINO"],
            axis=0)

In [ ]:
# Contamos el numero de filas resultante.
data["LATITUD_DESTINO"].count()

Ahora, haremos algunos leves ajustes y preparaciones para construir nuestra visualización. En primer lugar, definiremos una variable con la posición del centroide de la ciudad de Bogotá, que nos será de utilidad más adelante.

In [ ]:
# Las coordenadas geográficas centrales de Bogotá.

Bog_lat_long = (4.624335, -74.063644)
Bog_lat_long

Después, revisando los datos del *dataset* nos damos cuenta de una inconsistencia entre el formato almacenado y las latitudes y longitudes reales. Por ejemplo:

In [ ]:
# Observamos algunos de los datos de latitud y longitud del dataset.
data.loc[0, 'LATITUD_ORIGEN'], data.loc[0, 'LONGITUD_ORIGEN']

Como vemos, los datos en el dataset no se encuentran correctamente formateados, así que los formateamos correctamente. Aplicando conocimiento del dominio en datos espaciales es posible identificar que algunos valores son incorrectos y corresponden a coordenadas fuera de Bogotá. Inicialmente, veremos estadísticas descriptivas de las coordenadas para ver algunos posibles valores atípicos.

In [ ]:
data[["LATITUD_ORIGEN", "LATITUD_DESTINO", "LONGITUD_ORIGEN", "LONGITUD_DESTINO"]].describe()

Algunos valores parecen correctos, pero están en una escala incorrecta; mientras que otros, son claramente valores atípicos, como en el valor $0$ identificado en el mínimo de la variable de latitud del destino o en el máximo de la variable de longitud destino.

Gracias al conocimiento del dominio, decidimos limitar las coordenadas de la siguiente forma, dando un poco de margen para el área metropolitana de la ciudad:

* **Latitud:** Valores entre $4.4$ y $5.0$, dando un margen para viajes a municipios aledaños como Chipaque, Chia o Cajicá.
* **Longitud:** Valores entre $-74.3$ y $-73.9$, dando un margen para viajes a municipios aledaños como Madrid, Mosquera, Choachí o La Calera.

Ahora convertiremos y filtraremos los datos obtenidos para obtener los valores apropiados.

In [ ]:
#Filtramos los valores fuera del área

n_antes = len(data)

data = data[data['LATITUD_ORIGEN'].astype('str').str[0].isin(['4', '5']) &
            data['LATITUD_DESTINO'].astype('str').str[0].isin(['4', '5']) &
            data['LONGITUD_ORIGEN'].astype('str').str[:3].isin(['-73', '-74']) &
            data['LONGITUD_DESTINO'].astype('str').str[:3].isin(['-73', '-74'])]

n_despues = len(data)

print(f'{n_antes - n_despues} datos inconsistentes o fuera del área Metropolitana de Bogotá')

Ahora convertimos todos los valores a la misma escala que las coordenadas geográficas de Bogotá. Se multiplica cada valor por 10 elevado a la potencia de $n$, donde $n$ es el valor truncado del logaritmo base 10 de cada valor. 

Esta operación se realiza para asegurarnos de mantener el mismo número de dígitos enteros, evitando así inconsistencias entre valores en bases distintas.

In [ ]:
for variable in ['LATITUD_ORIGEN', 'LATITUD_DESTINO']:
  data[variable] = data[variable] * 10 ** -np.trunc(np.log10(data[variable]))

for variable in ['LONGITUD_ORIGEN', 'LONGITUD_DESTINO']:
  data[variable] = data[variable] * 10 ** -np.trunc(np.log10(-data[variable]) - 1)

Ahora comprobamos los rangos de las variables corregidas:

In [ ]:
data[['LATITUD_ORIGEN', 'LATITUD_DESTINO', 'LONGITUD_ORIGEN', 'LONGITUD_DESTINO']].agg([np.min, np.max]).T

In [ ]:
# Distribución de las latitudes.
px.box(data, y = ['LATITUD_ORIGEN', 'LATITUD_DESTINO'])

In [ ]:
# Distribución de las longitudes.
px.box(data, y = ['LONGITUD_ORIGEN', 'LONGITUD_DESTINO'])

Los datos obtenidos se encuentran ahora en rangos aceptables de valores. Podemos proceder y generar los mapas correspondientes usando estos valores geográficos.

### **2.3. Mapas de *Folium***
---
Ya con nuestros datos previamente seleccionados y preparados, podemos empezar a generar los mapas con *Leaflet* por medio de *Folium*. Para esto, usaremos el método **`folium.Map`**. Primero, creemos un mapa centrado en la ciudad de Bogotá.



In [ ]:
# Creamos nuestro primer Mapa con Folium con los datos de la primera encuesta.

test_map = folium.Map(location=Bog_lat_long,   # Posición inicial del mapa.
                      zoom_start=11)           # Zoom inicial del mapa. 

# Visualizamos el mapa de Bogotá
test_map

> **Nota**: Intente hacer zoom para conocer el nivel de detalle que puede alcanzar.

Con el método **`folium.Marker`** podemos crear marcadores que se mostrarán en nuestra visualización. Inicialmente, crearemos uno con el primer dato geográfico.

In [ ]:
# Los marcadores son de dos tipos principales. Los primeros son marcadores simples como el siguiente:

folium.Marker(location= Bog_lat_long,       # Definimos la posición geográfica del marcador.
              popup="Marcador simple.",     # Añadimos un texto desplegado al hacer clic en el marcador.
              tooltip="Haga clic!"          # Añadimos un texto desplegado al pasar el mouse encima del marcador. Sirve como sugerencia.
              ).add_to(test_map)            # Con la función .add_to podemos agregar el marcador a un mapa creado previamente.

test_map

 También podremos tener marcadores en forma de burbuja, generados con la función **`folium.vector_layers.CircleMarker`**. Estos pueden ser personalizados mediante aspectos como el color o el tamaño.

In [ ]:
# Marcadores de tipo burbuja.

folium.vector_layers.CircleMarker(location= (4.635, -74.084),          # Posición del marcador.
                                  fill=True,                           # Se puede definir si se colorea o no el marcador.
                                  fill_color="blue",                   # Color del marcador (azul).
                                  radius=20,                           # Radio del marcador.
                                  popup="Marcador circular",           # Texto desplegado al hacer clic en el marcador.
                                  tooltip="Otro marcador").add_to(test_map)

test_map

**Mapa de orígenes**
***

Crear los marcadores individualmente no es la decisión más apropiada para conjuntos muy grandes. Para hacer esto podemos optar por definir una función que realice esta tarea para cada punto dado.

In [ ]:
# Creamos un mapa nuevo.
origen_map = folium.Map(location=Bog_lat_long, zoom_start = 12)

Podemos definir una función que genere y añada el mapa un marcador para luego iterar sobre nuestra información.

In [ ]:
# Visualizamos los orígenes-destino en el mapa.

def setPoint(point, map, icon_color, **kwargs): # Esta notación permite obtener cualquier cantidad de argumentos y agruparlos en la variable 'kwargs'.

  popup_text = "" # Vamos a crear un texto compuesto para despegarlo como pop-up.
  
  # Usamos los valores en cada argumento adicional para componer el texto del mensaje.
  for k,v in kwargs.items():
    popup_text += f"{k}: {v}\n"
    
  # Definimos un marcador
  folium.Marker(location=point,
                popup=popup_text, 
                icon = folium.Icon(color=icon_color, icon='ok-sign')  # Podemos utilizar iconos de Leaflet para nuestros marcadores.
                ).add_to(map) # Finalmente lo agregamos al mapa entregado como argumento.

In [ ]:
# Dibujar los puntos en el mapa con datos con información en el popup para visualizar mejor lo limitamos a 50 ejemplos

for i in list(data.index)[:50]: # Iteramos el índice del DataFrame.
  setPoint(point=(data.loc[i, "LATITUD_ORIGEN"], data.loc[i, "LONGITUD_ORIGEN"]), # Posición del marcador.
           map=origen_map,                                                        # Mapa al que agregar el marcador.
           icon_color="green",                                                    # Color del marcador.
           # Argumentos adicionales usados como texto:
           motivo_viaje=data.loc[i, "MOTIVOVIAJE"], 
           hora_inicio=data.loc[i,"HORA_INICIO"], 
           hora_fin=data.loc[i, "HORA_FIN"], 
           medio=data.loc[i, "MEDIO_PREDOMINANTE"])
  
# Visualizamos el mapa
origen_map

Al igual que con *Plotly*, es posible generar un archivo *HTML* con las gráficas que generemos con *Folium*. Para esto se utiliza el método **`save`** en los mapas generados.

In [ ]:
# Guardamos el mapa
origen_map.save("Origen_map.html")

### **2.4. *ClusterMap***
---
Como pudimos observar en el gráfico superior, hay bastantes puntos de origen y destino por lo que visualizarlos resulta complicado y poco claro. En estos casos es apropiado agruparlos de acuerdo a sus coordenadas en *clusters* usando un ***Cluster map***. Esto es posible con la función **`FastMarkerCluster`**, que acepta como parámetro los datos completos y genera los *cluster* apropiados con respecto al acercamiento o *zoom* del mapa.

Visualicemos los destinos de viaje por medio de *clusters*:

In [ ]:
# Creamos un mapa nuevo
cluster_destino = folium.Map(location = Bog_lat_long,
                            zoom_start = 12,
                            control_scale = True
                            )

# Creamos tuplas con los valores de coordenadas.
destino_coords = data[['LATITUD_DESTINO', 'LONGITUD_DESTINO']].values

# Utilizamos los datos de origen para generar clusters dispuestos por la ciudad
FastMarkerCluster(name ="Cluster de coordenadas" ,
                  data= destino_coords).add_to(cluster_destino)

cluster_destino

Igual que antes, podemos exportar este mapa a *HTML*:

In [ ]:
cluster_destino.save("mapa_con_cluster_destino.html")

### **2.5. Heatmap**
---

Además de los mapas de *clusters*, podemos crear **mapas de calor** o ***heatmaps*** para visualizar los orígenes en las horas pico de la mañana. Para esto, usaremos las funcionalidades de datos temporales de *pandas* para seleccionar los datos que correspondan con el momento deseado.

In [ ]:
# Tiempo entre 6:00 am y 9:00 am

inicio = pd.to_datetime("06:00")  
fin = pd.to_datetime("09:00")

In [ ]:
# Convertimos las horas de inicio y fin de cadenas de texto a valores temporales.

data["HORA_INICIO"] = pd.to_datetime(data["HORA_INICIO"], errors='ignore')
data["HORA_FIN"] = pd.to_datetime(data["HORA_FIN"], errors='ignore')

In [ ]:
# Generamos los valores entre las horas definidas previamente usando selección condicional.

hora_pico_am = data[(data["HORA_INICIO"]>inicio) & (data["HORA_INICIO"]<fin)]

Ahora, generamos los datos de entrada del mapa de calor: 

In [ ]:
# Obtenemos las coordenadas que usaremos como entrada de la función HeatMap.
heat_data = hora_pico_am[['LATITUD_ORIGEN',  'LONGITUD_ORIGEN']]

heat_data.head(10)

Una vez obtenemos los datos de las posiciones de los datos de origen, podemos utilizar la función **`HeatMap`** de *Folium* para generar un mapa de calor con los valores, representando en los lugares con colores más cálidos las concentraciones mayores de datos. Este mapa de calor se ajusta dinámicamente según navegamos, ofreciendo más información al acercarse o alejarse de un lugar en particular.

In [ ]:
# Mapa de Folium. 
heatmap_bog = folium.Map(location=Bog_lat_long,
                          zoom_start = 16) 

# Añadimos los datos al mapa de Calor
HeatMap(heat_data.values).add_to(heatmap_bog)

#Visualizamos el mapa
heatmap_bog

In [ ]:
# Podemos guardar el archivo en forma de archivo HTML.
heatmap_bog.save("mapa_con_heat_map.html")

### **2.6. Ideas adicionales**
---

Con estas librerías podemos realizar varias visualizaciones adicionales muy útiles como las planteadas a continuación:
* Aplicar un filtro en el *dataset* de movilidad para seleccionar aquellos viajes realizados en transporte público y visualizarlos.
* Utilizar los datos de los polígonos que conforman las localidades (disponibles en este [enlace](https://bogota-laburbano.opendatasoft.com/explore/dataset/poligonos-localidades/export/?flg=es&location=8,4.2841,-74.21816&basemap=jawg.streets)) y agrupar los viajes de acuerdo con la localidad de origen y la localidad de destino y así visualizar los viajes interlocalidades.
* Filtrar los datos de acuerdo a la hora pico de la tarde o la mañana y agruparlos para determinar cuáles son los orígenes y destinos de estas horas. 
* Utilizar los datos geográficos de las localidades para realizar la visualización de un mapa coroplético de acuerdo al número de viajes, utilizando el siguiente [ejemplo](https://python-graph-gallery.com/292-choropleth-map-with-folium/) como guía.



# **Recursos adicionales**
---
* [Plotly - Python Open Source Graphing Library Maps](https://plotly.com/python/maps/)
* [Plotly - Choropleth Maps in Python](https://plotly.com/python/choropleth-maps/)
* [Medium (Dr. Dataman) - Create Beautiful Geomaps with Plotly](https://medium.com/analytics-vidhya/plotly-for-geomaps-bb75d1de189f)
* [Saiteja Kura - *Data Visualisation using Pandas and Plotly* - Medium.com](https://medium.com/towards-artificial-intelligence/data-visualisation-using-pandas-and-plotly-970df88fba6f) 
* [Duncan Parkes - Folium Marker Clusters](https://deparkes.co.uk/2016/06/24/folium-marker-clusters/)
* [Kaggle - How to: Folium for maps, heatmaps & time data](https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-data)
* [Folium - Quickstart / Getting Started](https://python-visualization.github.io/folium/quickstart.html#Getting-Started)
* [Jose Luis García Grandes - Folium: utilizando Leaflet con Python](https://mappinggis.com/2018/10/folium-utilizando-leaflet-con-python/)
* [Leaflet - Using GeoJSON with Leaflet](https://leafletjs.com/examples/geojson/)
* [Duncan Parkes - Plot Lines in Folium](https://deparkes.co.uk/2016/06/03/plot-lines-in-folium/)
* [Rob Story - Folium Circle Markers](http://bl.ocks.org/wrobstory/5609747)



# **Créditos**
---

* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistente docente:** Alberto Nicolai Romero Martínez

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*